Drag Models


In [ ]:
import requests
def download_file_url(url, local_filename):
    from java.io import File
    #mianly used to download the  solfsmy and dtc files for JB2008
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return File(local_filename)

# Download SOLFSMY and DTCFILE files for JB2008 model
solfsmy_file = download_file_url("https://sol.spacenvironment.net/JB2008/indices/SOLFSMY.TXT", "external/jb08_inputs/SOLFSMY.TXT")
dtcfile_file = download_file_url("https://sol.spacenvironment.net/JB2008/indices/DTCFILE.TXT", "external/jb08_inputs/DTCFILE.TXT")

from org.orekit.data import DataSource
# Create DataSource instances
solfsmy_data_source = DataSource(solfsmy_file)
dtcfile_data_source = DataSource(dtcfile_file)


In [ ]:
from orekit.pyhelpers import datetime_to_absolutedate
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import NadirPointing
from org.orekit.frames import FramesFactory
from org.orekit.utils import IERSConventions, PVCoordinates
from org.orekit.models.earth import ReferenceEllipsoid
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid
from org.orekit.orbits import CartesianOrbit
from org.hipparchus.ode.nonstiff import DormandPrince853Integrator
from orekit import JArray_double
from org.orekit.orbits import OrbitType
from org.orekit.forces.gravity.potential import GravityFieldFactory, TideSystem
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel, SolidTides, OceanTides, ThirdBodyAttraction, Relativity, NewtonianAttraction
# from org.orekit.forces import BoxAndSolarArraySpacecraft
from org.orekit.forces.radiation import SolarRadiationPressure, IsotropicRadiationSingleCoefficient, KnockeRediffusedForceModel
from org.orekit.forces.drag import DragForce, IsotropicDrag
from org.orekit.propagation.numerical import NumericalPropagator
from org.orekit.propagation import SpacecraftState
from org.orekit.utils import Constants
from org.orekit.models.earth.atmosphere.data import JB2008SpaceEnvironmentData, CssiSpaceWeatherData
from org.orekit.models.earth.atmosphere import JB2008, DTM2000, NRLMSISE00
from org.orekit.data import DataSource
from org.orekit.time import TimeScalesFactory   

In [ ]:
cross_section = 20.0
cd = 2.2

wgs84Ellipsoid = ReferenceEllipsoid.getWgs84(FramesFactory.getITRF(IERSConventions.IERS_2010, False))
jb08_data = JB2008SpaceEnvironmentData(solfsmy_data_source,
                                    dtcfile_data_source)
utc = TimeScalesFactory.getUTC()
sun = CelestialBodyFactory.getSun()
atmosphere = JB2008(jb08_data, sun, wgs84Ellipsoid, utc)
drag_sensitive = IsotropicDrag(float(cross_section), float(cd))
dragForce = DragForce(atmosphere, drag_sensitive)
atmospheric_drag_eci_t0 = extract_acceleration(state_vector, epochDate, mass, dragForce)
atmospheric_drag_eci_t0 = np.array([atmospheric_drag_eci_t0[0].getX(), atmospheric_drag_eci_t0[0].getY(), atmospheric_drag_eci_t0[0].getZ()])
accelerations_dict['jb08drag'] = atmospheric_drag_eci_t0
